In [ ]:
#
# for future me -- here are some commands that may help if cairo backend is not working...
#
#1735  sudo apt-get install cmake
#1739  sudo apt-get install python3-cairo
#1741  sudo apt-get install libcairo2
#1742  sudo apt-get install libcairo2-dev
#1743  pip install pycairo
#1744  pip install rlPyCairo

import pandas as pd
import polars as pl
import numpy as np
import time
from rtsvg import *
rt = RACETrack()
ts1 = time.time()
df = pl.concat([pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv'),
                pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk2.csv'),
                pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk3.csv')])
df = rt.columnsAreTimestamps(df, 'parsedDate')
ts2 = time.time()
print(f'Loading Time ... {ts2 - ts1:0.2} sec')

df = df.drop(['TimeSeconds', #'parsedDate',
              'dateTimeStr', #'ipLayerProtocol',
              'ipLayerProtocolCode', #'firstSeenSrcIp', #'firstSeenDestIp', #'firstSeenSrcPort', #'firstSeenDestPort',
              'moreFragments', 'contFragments', #'durationSeconds',
              'firstSeenSrcPayloadBytes', 'firstSeenDestPayloadBytes', #'firstSeenSrcTotalBytes', #'firstSeenDestTotalBytes', #'firstSeenSrcPacketCount', #'firstSeenDestPacketCount',
              'recordForceOut'])
df = df.rename({'parsedDate':'ts', 'ipLayerProtocol':'pro', 'firstSeenSrcIp':'sip', 'firstSeenDestIp':'dip', 'firstSeenSrcPort':'spt',
                'firstSeenDestPort':'dpt', 'durationSeconds':'dur', 'firstSeenSrcTotalBytes':'soct', 'firstSeenDestTotalBytes':'doct', 
                'firstSeenSrcPacketCount':'spkt', 'firstSeenDestPacketCount':'dpkt'})
df.sample(3)

In [ ]:
params = {'dot_size':'tiny', 'color_by':'dpt', 'w':512, 'h':256}
_xy0_ = rt.xy(df, x_field='ts', y_field='sip', **params)
_xy1_ = rt.xy(df, x_field='ts', y_field='dip', **params)
rt.tile([_xy0_, _xy1_])

In [3]:
_xy0_img_ = rt.displaySVGAsImage(_xy0_)
_xy1_img_ = rt.displaySVGAsImage(_xy1_)

In [ ]:
#
# From the following:
# https://huggingface.co/Salesforce/xgen-mm-phi3-mini-instruct-interleave-r-v1.5/blob/main/demo.ipynb
#
from transformers import AutoModelForVision2Seq, AutoTokenizer, AutoImageProcessor, StoppingCriteria
import torch
import PIL
import textwrap
import IPython.display as display
from IPython.display import Image

model_name_or_path = "Salesforce/xgen-mm-phi3-mini-instruct-interleave-r-v1.5"
model = AutoModelForVision2Seq.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True, use_fast=False, legacy=False)
image_processor = AutoImageProcessor.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = model.update_special_tokens(tokenizer)
model = model.to('cuda')
model.eval()
tokenizer.padding_side = "left"
tokenizer.eos_token = '<|end|>'
def apply_prompt_template(prompt):
    s = (
                '<|system|>\nA chat between a curious user and an artificial intelligence assistant. '
                "The assistant gives helpful, detailed, and polite answers to the user's questions.<|end|>\n"
                f'<|user|>\n{prompt}<|end|>\n<|assistant|>\n'
            )
    return s 

In [ ]:
image_list  = [] # the image_processor converts it to a tensor (torch.Tensor)
                 # the input that works in the example is PIL.JpegImagePlugin.JpegImageFile
image_sizes = [] # tuples (w,h)
image_list.append(image_processor([_xy0_img_], image_aspect_ratio='anyres')["pixel_values"].cuda())
image_sizes.append((_xy0_img_.width, _xy0_img_.height))
inputs = { 'pixel_values': [image_list]}

In [ ]:
#import torchvision.transforms as transforms
#transform = transforms.Compose([transforms.ToTensor()])
#transform(_xy0_img_)